In [7]:
import math

import objaverse
import random

uids = objaverse.load_uids() # each object has a unique ID (uid), then load it
print("all of objects length :", len(uids))

random.seed(42)
lvis_annotations = objaverse.load_lvis_annotations()

num_of_objects = 50
filtered_lvis_annotations = {key: value for key, value in lvis_annotations.items() if len(value) >= num_of_objects}
print(f"the number of classes having items more than {num_of_objects} : {len(filtered_lvis_annotations)}")

all of objects length : 798759
the number of classes having items more than 50 : 319


In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

my_datasets = {}
count = 1
for key, value in filtered_lvis_annotations.items():
    sampled_uids = random.sample(value, num_of_objects)
    my_datasets[key] = sampled_uids
    
    print(f"\nDownloading {key}... ({count} / {len(filtered_lvis_annotations)})")
    objaverse.load_objects(sampled_uids, 1)
    print(f"Downloading {key} done.\n")
    count += 1

print("All downloading are done.")


Downloaded 1 / 50 objects
Downloaded 2 / 50 objects
Downloaded 3 / 50 objects
Downloaded 4 / 50 objects
Downloaded 5 / 50 objects
Downloaded 6 / 50 objects
Downloaded 7 / 50 objects
Downloaded 8 / 50 objects
Downloaded 9 / 50 objects
Downloaded 10 / 50 objects
Downloaded 11 / 50 objects
Downloaded 12 / 50 objects
Downloaded 13 / 50 objects
Downloaded 14 / 50 objects
Downloaded 15 / 50 objects
Downloaded 16 / 50 objects
Downloaded 17 / 50 objects
Downloaded 18 / 50 objects
Downloaded 19 / 50 objects
Downloaded 20 / 50 objects
Downloaded 21 / 50 objects
Downloaded 22 / 50 objects
Downloaded 23 / 50 objects
Downloaded 24 / 50 objects
Downloaded 25 / 50 objects
Downloaded 26 / 50 objects
Downloaded 27 / 50 objects
Downloaded 28 / 50 objects
Downloaded 29 / 50 objects
Downloaded 30 / 50 objects
Downloaded 31 / 50 objects
Downloaded 32 / 50 objects
Downloaded 33 / 50 objects
Downloaded 34 / 50 objects
Downloaded 35 / 50 objects
Downloaded 36 / 50 objects
Downloaded 37 / 50 objects
Downloade

In [ ]:
import torch

# Local GPU Test

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

In [ ]:
import json
import trimesh
import gzip
import pyrender
import numpy as np
import os
from PIL import Image

class Renderer:
    def __init__(self, path_file, output_path):
        self.my_datasets = my_datasets # key : a class, value : a list of uids
        self.base_path = os.path.join("D:\Sehyeon\Datasets\objaverse", "hf-objaverse-v1")
        with gzip.open(path_file, 'rt') as f:
            self.object_paths = json.load(f)
        self.output_path = output_path
        
    def load_mesh(self, uid):
        # load 3D .glb mesh from its UID
        mesh_path = os.path.join(self.base_path, self.object_paths[uid])
        if not mesh_path or not os.path.exists(mesh_path):
            raise FileNotFoundError(f"Model file not found for UID: {uid}")
        mesh = trimesh.load(mesh_path)
        
        # normalize the mesh to fit inside a 2*2*2 cube centered at (0,0,0)
        centroid = mesh.bounds.mean(axis=0)
        mesh.apply_translation(-centroid)
        
        max_extent = mesh.extents.max()
        scale_factor = 2.0 / max_extent
        mesh.apply_scale(scale_factor)
        
        return mesh
    
    def render_mesh(self, mesh, angles=[45, 135, 225, 315], distance=2.5, image_size=(512, 512)):
        scene = pyrender.Scene()
        mesh = pyrender.Mesh.from_trimesh(mesh)
        scene.add(mesh)
        
        camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
        light = pyrender.DirectionalLight(color=np.array([1.0, 1.0, 1.0]), intensity=3.0)
        
        images = []
        
        for angle in angles:
            # set camera pose
            camera_pose = np.array([
                [np.cos(np.radians(angle)), 0, np.sin(np.radians(angle)), distance * np.cos(np.radians(angle))],
                [0, 1, 0, 0],
                [-np.sin(np.radians(angle)), 0, np.cos(np.radians(angle)), distance * -np.sin(np.radians(angle))],
                [0, 0, 0, 1],
            ])
            scene.add(camera, pose=camera_pose)
            scene.add(light, pose=camera_pose)
            
            # render the scene
            r = pyrender.OffscreenRenderer(*image_size)
            color, _ = r.render(scene)
            images.append(Image.fromarray(color))
            
            # clean up the camera and light for the next angle
            scene.remove_node(scene.get_nodes(name=camera.name)[0])
            scene.remove_node(scene.get_nodes(name=light.name)[0])
        
        return images
    
    def process_dataset(self):
        for cls, uids in self.my_datasets.items():
            class_dir = os.path.join(self.output_path, cls)
            os.makedirs(class_dir, exist_ok=True)
            
            for uid in uids:
                try:
                    mesh = self.load_mesh(uid)
                    images = self.render_mesh(mesh)
                    
                    for i, img in enumerate(images):
                        img_path = os.path.join(class_dir, f"{uid}_{i}.png")
                        img.save(img_path)
                        print(f"Saved {img_path}")
                        
                except Exception as e:
                    print(f"Error processing UID {uid}: {e}")
        
        print("Rendering completed!")